In [30]:
import tensorflow as tf
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import os

mulu='E://aa//'
classes={'husky':1,'chihuahua':2}#分为两类
writer=tf.python_io.TFRecordWriter("dog_train.tfrecords")#要生成的文件
for index,name in enumerate(classes):
    class_path=mulu+name+'//'
    for img_name in os.listdir(class_path):#读取目录下的图片
        img_path=class_path+img_name#每一张图片的地址
        img=Image.open(img_path)#打开每一张图片
        img=img.resize((128,128))
        img_raw=img.tobytes()#将图片转化为二进制格式
        example=tf.train.Example(
            features=tf.train.Features(
                feature={
                    "label":tf.train.Feature(int64_list=tf.train.Int64List(value=[index])),
                    'img_raw': tf.train.Feature(bytes_list=tf.train.BytesList(value=[img_raw]))
                }
            )
        )
        writer.write(example.SerializeToString())
        
writer.close()

In [25]:
def read_and_decode(filename):
    filename_queue=tf.train.string_input_producer([filename])#生成一个队里
    reader=tf.TFRecordReader()
    _,serialized_example=reader.read(filename_queue)#返回文件名和文件
    #将image数据和label取出来
    features=tf.parse_single_example(serialized_example,
                                     features={
                                        'label': tf.FixedLenFeature([], tf.int64),
                                        'img_raw' : tf.FixedLenFeature([], tf.string),
                                     }
                                    )
    img=tf.decode_raw(features['img_raw'],tf.uint8)
    img=tf.reshape(img,[128,128,3])
    img=tf.cast(img,tf.float32)*(1./255)-0.5#在流中抛出img张量
    label=tf.cast(features['label'],tf.int32)#在流中抛出label张量
    return img,label


In [26]:
image,label=read_and_decode("dog_train.tfrecords")

In [28]:
print(image)
print(label)

Tensor("sub_2:0", shape=(128, 128, 3), dtype=float32)
Tensor("Cast_7:0", shape=(), dtype=int32)


In [29]:
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    coord=tf.train.Coordinator()
    threads=tf.train.start_queue_runners(coord=coord)
    for i in range(20):
        #在会话中提取image和label
        example,l=sess.run([image,label])
        img=Image.fromarray(example,'RGB')
        img.save('E://aa//'+str(i)+'_''Label_'+str(l)+'.jpg')#存下图片
        #print(example,1)
    coord.request_stop()
    coord.join(threads)

<tf.Tensor 'sub:0' shape=(128, 128, 3) dtype=float32>

In [1]:
import os 
import tensorflow as tf 
from PIL import Image  #注意Image,后面会用到
import matplotlib.pyplot as plt 
import numpy as np
 
cwd='E://aa//'
classes={'husky','chihuahua'} #人为 设定 2 类
writer= tf.python_io.TFRecordWriter("dog_train.tfrecords") #要生成的文件
 
for index,name in enumerate(classes):
    class_path=cwd+name+'\\'
    for img_name in os.listdir(class_path): 
        img_path=class_path+img_name #每一个图片的地址
 
        img=Image.open(img_path)
        img= img.resize((512,512))
        img_raw=img.tobytes()#将图片转化为二进制格式
        example = tf.train.Example(features=tf.train.Features(feature={
            "label": tf.train.Feature(int64_list=tf.train.Int64List(value=[index])),
            'img_raw': tf.train.Feature(bytes_list=tf.train.BytesList(value=[img_raw]))
        })) #example对象对label和image数据进行封装
        writer.write(example.SerializeToString())  #序列化为字符串
writer.close()

In [2]:
filename_queue = tf.train.string_input_producer(["dog_train.tfrecords"]) #读入流中
reader = tf.TFRecordReader()
_, serialized_example = reader.read(filename_queue)   #返回文件名和文件
features = tf.parse_single_example(serialized_example,
                                   features={
                                       'label': tf.FixedLenFeature([], tf.int64),
                                       'img_raw' : tf.FixedLenFeature([], tf.string),
                                   })  #取出包含image和label的feature对象
image = tf.decode_raw(features['img_raw'], tf.uint8)
image = tf.reshape(image, [512, 512, 3])
label = tf.cast(features['label'], tf.int32)
with tf.Session() as sess: #开始一个会话
    init_op = tf.initialize_all_variables()
    sess.run(init_op)
    coord=tf.train.Coordinator()
    threads= tf.train.start_queue_runners(coord=coord)
    for i in range(20):
        example, l = sess.run([image,label])#在会话中取出image和label
        img=Image.fromarray(example, 'RGB')#这里Image是之前提到的
        img.save(cwd+str(i)+'_''Label_'+str(l)+'.jpg')#存下图片
        #print(example, l)
    coord.request_stop()
    coord.join(threads)

Instructions for updating:
Use `tf.global_variables_initializer` instead.
